# Imports and setup

In [ ]:
import pyvista as pv
import meshio
import numpy as np
import psycopg2
import hdf5storage
import h5py
import pickle
import logging
from sys import getsizeof
from dotenv import dotenv_values
import matplotlib.pyplot as plt

pv.global_theme.transparent_background = True

config = dotenv_values("../.env")
# Filepaths for script
initMeshPath = config["currentDirectory"] +"data/visualizations/tempMeshes/Trial757_step0.vtu"
datasetPath = config["datasetFilepath"]
outputPath = config["currentDirectory"] + "src/datasetSubmission/Videos/"


In [2]:
# Read in dataset HDF5 file and look at structure
with h5py.File(datasetPath, 'r') as f:
    print("Keys: %s" % f.keys())
    X_fom = f["stateData"][:,0,0]
    print("Shape of X_fom: ", X_fom.shape)
    u_fom = f["inputData"][:,0,0]
    print("Shape of u_fom: ", u_fom.shape)
    outputData = f["outputData"][:,0,0]
    print("Shape of outputData: ", outputData.shape)
    time = f["simulationTimes"]
    print("Shape of time: ", time.shape)

Keys: <KeysViewHDF5 ['inputData', 'outputData', 'simulationTimes', 'simulationTimesteps', 'stateData']>
Shape of X_fom:  (243789,)
Shape of u_fom:  (6,)
Shape of outputData:  (40,)
Shape of time:  (1000, 40)


In [3]:
# Trial Parameters
trial_ids = [757]
timesteps = 1000
trial_id = trial_ids[0]


In [4]:
# Read in a mesh to get structure
initMesh = pv.read(initMeshPath)
nNodes = initMesh.n_points
nElements = initMesh.n_cells
initMeshFaces = initMesh.cells
print(f"Mesh has {nNodes} nodes and {nElements} elements.")
print(initMeshFaces)

Mesh has 81263 nodes and 721784 elements.
[    3     0  1220 ...  2248 76928 63807]


In [5]:
initMeshPoints = initMesh.points.copy()
initMeshCells = initMesh.cells.copy()

In [6]:
initMeshPoints.shape

(81263, 3)

In [7]:
# Reshape data to see what permutations look like
X_fom_reshaped = X_fom.reshape((nNodes, 3))
X_fom_reshaped.shape

(81263, 3)

In [8]:
print(initMeshPoints[0:10,:])
print(X_fom_reshaped[0:10,:])

[[-1.48000e+02 -5.21609e-05  4.99956e+01]
 [-1.48000e+02 -5.21609e-05 -4.99956e+01]
 [-1.48000e+02  2.69360e+00  4.99665e+01]
 [-1.48000e+02  8.33259e+01  2.00344e-06]
 [-1.48000e+02  2.69361e+00 -4.99665e+01]
 [-1.48000e+02 -2.69370e+00  4.99665e+01]
 [-1.48000e+02 -8.33260e+01  2.00335e-06]
 [-1.48000e+02 -2.69371e+00 -4.99665e+01]
 [-1.50000e+02 -5.25157e-05  4.99978e+01]
 [-1.50000e+02 -1.13656e+00  4.99915e+01]]
[[-1.48000000e+02 -5.21609373e-05  4.99955557e+01]
 [-1.48000000e+02 -5.21609368e-05 -4.99955531e+01]
 [-1.48000000e+02  2.69359719e+00  4.99665128e+01]
 [-1.48000000e+02  8.33258939e+01  2.00343703e-06]
 [-1.48000000e+02  2.69360841e+00 -4.99665098e+01]
 [-1.48000000e+02 -2.69370357e+00  4.99665127e+01]
 [-1.48000000e+02 -8.33259986e+01  2.00335194e-06]
 [-1.48000000e+02 -2.69371478e+00 -4.99665098e+01]
 [-1.50000000e+02 -5.25157013e-05  4.99977932e+01]
 [-1.50000000e+02 -1.13656134e+00  4.99915412e+01]]


In [9]:
# ### Get offsets for reference trajectory and vectorized mesh from database
# # Load data from database
# conn = get_db_connection()
# data = fetch_trial_data_singleTimestep(conn, trial_ids[0], 1, start_timestep=0, num_timesteps=1)
# conn.close()
# timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
# # Get the reference trajectory
# refOffsets = np.copy(output_data)


In [10]:
# state_data.shape

In [11]:
# test = initMeshPoints - state_data[0:nNodes, 0:3]

Ran the code below to figure out the permutation between the mesh data saved by sofa as .vtu files and the mesh points saved in the database

It turns out the mesh points saved in the database are perfectly ordered in the same way as the points in the .vtu file. So we'll just extract the face's defined in the .vtu file and use that as the faces at every time step.

If theres ever a point where the permutation mapping is in question though. Can uncomment the code below and find the permutation

In [12]:
# # Find permutation that maps initMeshPoints to state_data
# # This assumes that the points are the almost the same, just in different order
# # We use a tolerance to account for numerical differences
# permutation = []
# used_indices = set() # indices used so far in initial mesh .vtu
# for i, point in enumerate(initMeshPoints):
#     # Subtract point from all state_data points
#     diffs = state_data[:, 0:3] - point
#     dists = np.linalg.norm(diffs, axis=1)
#     # Find the index of the closest point in state_data that hasn't been used yet
#     min_index = np.argmin(dists)
#     # Check if this index has been used
#     while min_index in used_indices:
#         print("Collision detected, finding next closest point")
#         dists[min_index] = float('inf')
#         min_index = np.argmin(dists)
#     permutation.append(min_index)
#     used_indices.add(min_index)
#     if i % 1000 == 0:
#         print(f"Processed {i} points")
# permutation = np.array(permutation)
# print(f"Permutation found with length {len(permutation)} and unique indices {len(set(permutation))}")
# print(permutation)


# # Check that permutation is valid
# if len(permutation) != len(set(permutation)):
#     raise ValueError("Permutation is not valid, some indices are used multiple times")
# # Check that permutation maps initMeshPoints to state_data
# permuted_initMeshPoints = initMeshPoints[permutation]
# max_diff = np.max(np.linalg.norm(permuted_initMeshPoints - state_data[:, 0:3], axis=1))
# print(f"Max difference after permutation: {max_diff}")

# Render animated mesh of robot simulations for each time step

In [13]:
# # Stuff from other file for generating surfaces
#         #### Draw Surface for reference trajectory
#         # Number of samples (in depth dir) and number of points along centerline
#         ### Make surface from desired reference trajectory
#         # Load data from database
#         conn = get_db_connection()
#         data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
#         conn.close()
#         timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        
#         # Check shape of reference trajectory
#         uncenteredRefTraj = y_ref + refOffsets
#         # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
#         x_refs = uncenteredRefTraj[::2]
#         z_refs = uncenteredRefTraj[1::2]
#         referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
#         path = referenceCurve
#         offsetPath = np.copy(path)
#         offsetPath[:, 1] = offsetPath[:, 1] - 122
 
#         nsamples = 3
#         ntraces = 20  
#         # Define the Z spacing of your 2D section
#         z_spacing = 60
#         # Create structured points draping down from the path
#         points = np.repeat(path, nsamples, axis=0)
#         # repeat the Z locations across
#         tp = np.arange(0, z_spacing * nsamples, z_spacing)
#         # Offset the Z locations 
#         tp = path[:, 1][:, None] - tp
#         points[:, 1] = tp.ravel()
#         # Make a StructuredGrid from the structured points
#         grid = pv.StructuredGrid()
#         grid.points = points
#         grid.dimensions = nsamples, ntraces, 1
#         # Add the data array - note the ordering
#         grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")
        
#         ### Plot actual centerline from output_data
#         # Load data from database
#         conn = get_db_connection()
#         data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
#         conn.close()
#         timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        
#         # Check shape of output data
#         uncenteredOutputTraj = output_data
#         # Turn output trajectory from 40x1 to 20x2 array by taking every other point
#         x_outs = uncenteredOutputTraj[::2]
#         z_outs = uncenteredOutputTraj[1::2]
#         outputCurve = np.column_stack((x_outs, np.zeros(np.size(x_outs)), z_outs))
#         offsetOutputCurve = np.copy(outputCurve)
#         offsetOutputCurve[:, 1] = offsetOutputCurve[:, 1] - 122
        
#         # load mesh from .vtu file in tempMeshPath
#         mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{i}.vtu")
        
#         # Add meshes to top view window
#         plotter.subplot(0, 0)
#         plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
#         plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
#         plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name='refPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
#         plotter.add_mesh(pv.PolyData(offsetOutputCurve), color='blue', line_width=5, name='outputPath', opacity=opacities, render_points_as_spheres=True, point_size=10)

In [14]:
# Helper function to that returns pyvista mesh given a trial and timestep from the dataset 
def get_mesh_for_timestep(trial_id, timestep):
    with h5py.File(datasetPath, 'r') as f:
        # print("Keys: %s" % f.keys())
        state_data = f["stateData"][:,timestep,trial_id-1]
        # print("Shape of state_data: ", state_data.shape)
        input_data = f["inputData"][:,timestep,trial_id-1]
        # print("Shape of u_fom: ", u_fom.shape)
        output_data = f["outputData"][:,timestep,trial_id-1]
        # print("Shape of outputData: ", outputData.shape)
        time = f["simulationTimes"]
        # print("Shape of time: ", time.shape)
    # Reshape state_data
    state_data = state_data.reshape((nNodes, 3))
    # Create a new pyvista mesh with the updated points
    newMesh = pv.PolyData()
    newMesh.points = state_data[0:nNodes, 0:3]
    newMesh.faces = initMeshFaces

    # # Generate surface meshes for reference and output trajectories
    # uncenteredRefTraj = y_ref + refOffsets
    # # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
    # x_refs = uncenteredRefTraj[::2]
    # z_refs = uncenteredRefTraj[1::2]
    # referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
    # path = referenceCurve
    # offsetPath = np.copy(path)
    # offsetPath[:, 1] = offsetPath[:, 1] - 122

    # nsamples = 3
    # ntraces = 20  
    # # Define the Z spacing of your 2D section
    # z_spacing = 60
    # # Create structured points draping down from the path
    # points = np.repeat(path, nsamples, axis=0)
    # # repeat the Z locations across
    # tp = np.arange(0, z_spacing * nsamples, z_spacing)
    # # Offset the Z locations 
    # tp = path[:, 1][:, None] - tp
    # points[:, 1] = tp.ravel()
    # # Make a StructuredGrid from the structured points
    # grid = pv.StructuredGrid()
    # grid.points = points
    # grid.dimensions = nsamples, ntraces, 1
    # # Add the data array - note the ordering
    # grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")

    # ### Plot actual centerline from output_data    
    # # Check shape of output data
    # uncenteredOutputTraj = output_data
    # # Turn output trajectory from 40x1 to 20x2 array by taking every other point
    # x_outs = uncenteredOutputTraj[::2]
    # z_outs = uncenteredOutputTraj[1::2]
    # outputCurve = np.column_stack((x_outs, np.zeros(np.size(x_outs)), z_outs))
    # offsetOutputCurve = np.copy(outputCurve)
    # offsetOutputCurve[:, 1] = offsetOutputCurve[:, 1] - 122

    # ref_mesh = pv.PolyData(offsetPath)
    # output_mesh = pv.PolyData(offsetOutputCurve)


    return newMesh#, ref_mesh, output_mesh, grid


In [ ]:
# Render a single frame to check everything looks good

# mesh, ref_mesh, output_mesh, grid = get_mesh_for_timestep(trial_id, 0)
mesh = get_mesh_for_timestep(1, 0)

# Plot the mesh


plotter = pv.Plotter(shape=(2,1), border=False, row_weights = [0.4,1], window_size=[1008, 1008])

plotter.subplot(0, 0)
plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
# plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
# plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
# plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
plotter.camera_position = 'yz'
plotter.camera.roll = 180
plotter.camera.azimuth = 90
plotter.camera.elevation = 90
plotter.enable_parallel_projection()
# change bounds to be 0, 0, -200, 200, -200, 200
plotter.camera_position = [
    (-558.609577998519, -2226.5518997921977, -13.31930130655202),
    (-558.609577998519, -19.333351135253906, -13.31930130655202),
    (0.0, -1.0, -2.220446049250313e-16)]
plotter.camera.zoom(3)

# Add meshes to isometric view window
plotter.subplot(1, 0)
plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
# plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
# plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
# plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
plotter.camera_position = 'yz'
plotter.camera.roll = 180
plotter.camera.azimuth = 40
plotter.camera.elevation = 50
plotter.enable_parallel_projection()
plotter.camera_position = [
    (528.2333916563371, -1710.160855083042, 898.6502336784569),
    (-558.609577998519, -19.333351135253906, -13.31930130655202),
    (0.0, -1.0, -2.220446049250313e-16)]
plotter.camera.zoom(1.25)
plotter.show()  




Widget(value='<iframe src="http://localhost:36103/index.html?ui=P_0x7e48bdd11360_0&reconnect=auto" class="pyvi…

In [ ]:
# Helper function that renders video 
def render_video(trial_id, timesteps):
    # create plotter objects
    plotter = pv.Plotter(shape=(2,1), border=False, row_weights = [0.4,1], window_size=[1008, 1008])
    # Open a mp4
    plotter.open_movie(outputPath + f"EelRobotDataset_Trial{trial_id}_CombinedView.mp4", framerate=100, quality=7)
    print(f"Rendering video for Trial {trial_id}...")
    for i in range(timesteps):
        # mesh, ref_mesh, output_mesh, grid = get_mesh_for_timestep(trial_id, i)
        mesh = get_mesh_for_timestep(trial_id, i)
        # # Plot the mesh
        # if i % 100 == 0:
        #     print(f"Rendering frame {i+1} of {timesteps} for Trial {trial_id}")

        # plotter = pv.Plotter(shape=(2,1), border=False, row_weights = [0.4,1], window_size=[1000, 1000])

        plotter.subplot(0, 0)
        plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
        # plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
        # plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
        # plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 90
        plotter.camera.elevation = 90
        plotter.enable_parallel_projection()
        # change bounds to be 0, 0, -200, 200, -200, 200
        plotter.camera_position = [
            (-558.609577998519, -2226.5518997921977, -13.31930130655202),
            (-558.609577998519, -19.333351135253906, -13.31930130655202),
            (0.0, -1.0, -2.220446049250313e-16)]
        plotter.camera.zoom(3)

        # Add meshes to isometric view window
        plotter.subplot(1, 0)
        plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
        # plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
        # plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
        # plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 40
        plotter.camera.elevation = 50
        plotter.enable_parallel_projection()
        plotter.camera_position = [
            (528.2333916563371, -1710.160855083042, 898.6502336784569),
            (-558.609577998519, -19.333351135253906, -13.31930130655202),
            (0.0, -1.0, -2.220446049250313e-16)]
        plotter.camera.zoom(1.25)
        plotter.write_frame()
    # Close the gif
    plotter.close()

In [ ]:
trials = range(1,41)
timesteps = 1000
for trial in trials:
    render_video(trial, timesteps)


In [ ]:
# Render a single frame  with just iso view to check everything looks good

mesh = get_mesh_for_timestep(1, 0)

# Plot the mesh


plotter = pv.Plotter(border=False, window_size=[1008, 1008])

# Add meshes to isometric view window
plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
# plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
# plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
# plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
plotter.camera_position = 'yz'
plotter.camera.roll = 180
plotter.camera.azimuth = 40
plotter.camera.elevation = 50
plotter.enable_parallel_projection()
plotter.camera_position = [
    (528.2333916563371, -1710.160855083042, 898.6502336784569),
    (-558.609577998519, -19.333351135253906, -13.31930130655202),
    (0.0, -1.0, -2.220446049250313e-16)]
plotter.camera.zoom(1.25)
plotter.show()  




Widget(value='<iframe src="http://localhost:36103/index.html?ui=P_0x7e48bcbb3e20_1&reconnect=auto" class="pyvi…

In [ ]:
# Helper function that renders video for just iso view
def render_video_iso(trial_id, timesteps):
    # create plotter objects
    plotter = pv.Plotter(border=False, window_size=[1008, 1008])
    # Open a mp4
    plotter.open_movie(outputPath + f"EelRobotDataset_Trial{trial_id}_IsoView.mp4", framerate=100, quality=7)
    print(f"Rendering video for Trial {trial_id}...")
    for i in range(timesteps):
        # mesh, ref_mesh, output_mesh, grid = get_mesh_for_timestep(trial_id, i)
        mesh = get_mesh_for_timestep(trial_id, i)
        # Add meshes to isometric view window
        plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
        # plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
        # plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
        # plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 40
        plotter.camera.elevation = 50
        plotter.enable_parallel_projection()
        plotter.camera_position = [
            (528.2333916563371, -1710.160855083042, 898.6502336784569),
            (-558.609577998519, -19.333351135253906, -13.31930130655202),
            (0.0, -1.0, -2.220446049250313e-16)]
        plotter.camera.zoom(1.25)
        plotter.write_frame()
    # Close the gif
    plotter.close()

In [21]:
trials = range(1,41)
timesteps = 1000
for trial in trials:
    render_video_iso(trial, timesteps)

Rendering video for Trial 1...
Rendering video for Trial 2...
Rendering video for Trial 3...
Rendering video for Trial 4...
Rendering video for Trial 5...
Rendering video for Trial 6...
Rendering video for Trial 7...
Rendering video for Trial 8...
Rendering video for Trial 9...
Rendering video for Trial 10...
Rendering video for Trial 11...
Rendering video for Trial 12...
Rendering video for Trial 13...
Rendering video for Trial 14...
Rendering video for Trial 15...
Rendering video for Trial 16...
Rendering video for Trial 17...
Rendering video for Trial 18...
Rendering video for Trial 19...
Rendering video for Trial 20...
Rendering video for Trial 21...
Rendering video for Trial 22...
Rendering video for Trial 23...
Rendering video for Trial 24...
Rendering video for Trial 25...
Rendering video for Trial 26...
Rendering video for Trial 27...
Rendering video for Trial 28...
Rendering video for Trial 29...
Rendering video for Trial 30...
Rendering video for Trial 31...
Rendering video f